<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#下载文件" data-toc-modified-id="下载文件-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>下载文件</a></span><ul class="toc-item"><li><span><a href="#impala-脚本设置" data-toc-modified-id="impala-脚本设置-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>impala 脚本设置</a></span></li><li><span><a href="#下载数据" data-toc-modified-id="下载数据-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>下载数据</a></span></li></ul></li><li><span><a href="#上传文件" data-toc-modified-id="上传文件-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>上传文件</a></span></li></ul></div>

In [7]:
# import module
# impala
from __future__ import print_function
from impala.dbapi import connect
from impala.util import as_pandas

# sftp
import paramiko
import csv
import os.path
import pysftp

In [8]:
file_path = 'D:\\lijiaxiang\\180601_product\\proct_file_mgt\\66bank\\deployment\\test_upload_ljx.csv'
remote = 'upload/test02.csv'
#远程文件或目录，与本地一致，当前为linux目录格式

# 下载文件

## impala 脚本设置

In [80]:
def run_sql(sql):
    conn = connect(host='******', auth_mechanism='PLAIN', port=10000, user='lijiaxiang', password='******')
    cursor = conn.cursor()
    cnt = 1

    # 2.2 对impala执行SQL查询
    if ';' in sql:
        sql_list = sql.rstrip().split(';')
        # print(type(sql_list))
        if len(sql_list[-1]):
            for s in sql_list:
                print("runing sql @ %s" % cnt)
                cursor.execute(s)
                cnt += 1
        else:
            sql_list.pop()
            for s in sql_list:
                print("runing sql @ %s" % cnt)
                cursor.execute(s)
                cnt += 1
    else:
        print("runing sql @ %s" % cnt)
        cursor.execute(sql)
    # 2.3 把结果转化成pandas的dataframe格式，以便进行数据分析
    # df = as_pandas(cursor)
    # print(df)
    return as_pandas(cursor) if cursor.description != None else 'null'

In [1]:
# df = run_sql('select 2 as x1')
# 测试impala包运行成功

In [82]:
df.to_csv(file_path)

## 下载数据

# 上传文件

In [3]:
def sftp_upload(host,port,username,password,local,remote):
    sf = paramiko.Transport((host,port))
    sf.connect(hostkey = None, username = username,password = password)
    sftp = paramiko.SFTPClient.from_transport(sf)
    try:
        sftp.mkdir("test")
    except IOError:
        print("(assuming test/ already exists)")
    try:
        if os.path.isdir(local):#判断本地参数是目录还是文件
            for f in os.listdir(local):#遍历本地目录
                sftp.put(os.path.join(local+f),os.path.join(remote+f))#上传目录中的文件
        else:
            sftp.put(local,remote)#上传文件
    except Exception,e:
        print('upload exception:',e)
    sf.close()

In [9]:
host = '******'#主机
port = 10000 #端口
username = '******' #用户名
password = '******' #密码
local = '******'
#本地文件或目录，与远程一致，当前为windows目录格式，window目录中间需要使用双斜线

In [10]:
sftp_upload(host,port,username,password,local,remote) #上传

(assuming test/ already exists)
